<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/4_Finetuned_Generation_unsloth_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuned Predictions
In this file we generate the finetuned predictions

## Setup

In [1]:
!pip install unsloth_zoo
!pip install --no-deps unsloth
!pip install -U transformers
!pip install accelerate bitsandbytes
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.5.6 requires xformers>=0.0.27.post2, which is not installed.


In [2]:
import torch
from google.colab import userdata
import logging
import transformers
import os
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
from bitsandbytes import nn as bnb_nn
from unsloth import FastLanguageModel
from tqdm.auto import tqdm
import pandas as pd
import json
from collections import defaultdict
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

<ipython-input-2-22ccd326f167>:9: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
token = userdata.get('GITHUB')
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 1157, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 1157 (delta 169), reused 123 (delta 83), pack-reused 913 (from 1)
Receiving objects: 100% (1157/1157), 48.19 MiB | 16.79 MiB/s, done.
Resolving deltas: 100% (645/645), done.
Updating files: 100% (110/110), done.


In [4]:
os.chdir("NLP2025_CQG")
!ls

1_a_Generate_DPO_Dataset.ipynb	      Development
1_Information_preprocessing.md	      Doc
1_Preprocessing.ipynb		      Evaluation
2_Baseline_Generation.ipynb	      INFORMATION.md
2_Information_Baseline_Generation.md  LICENSE
3_Evaluation.ipynb		      Logs
3_Training_1_SFT_3.ipynb	      README.md
4_Finetuned_Generation.ipynb	      requirements.txt
4_Finetuned_Generation_unsloth.ipynb  Training
5_Evaluation_Analytics.ipynb	      Utils
Data


In [5]:
################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

BASE_MODEL = "unsloth/Meta-Llama-3.1-8B-Instruct"
model_name= "Meta-Llama-3.1-1B-Instruct_DPO"
model_id= f"unsloth/Meta-Llama-3.1-8B-Instruct"

test_dataset_path = f"Data/Processed/test.csv"

results_path = os.path.join(os.getcwd(), f"Evaluation/Results/results_{model_name}.json")

log_base_path = f"Logs/"
os.makedirs(log_base_path, exist_ok=True)

log_path = log_base_path + "4_cqs_generation_SFT_2.log"


################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################


In [6]:
# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

# Log the device info
logger.info("--------  Start with Baseline Generation  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Results Path: {results_path}')
logger.info(f'Log Path: {log_path}')
logger.info("--------------------------------------------------------")

INFO:__main__:--------  Start with Baseline Generation  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Results Path: /content/NLP2025_CQG/Evaluation/Results/results_Meta-Llama-3.1-1B-Instruct_DPO.json
INFO:__main__:Log Path: Logs/4_cqs_generation_SFT_2.log
INFO:__main__:--------------------------------------------------------


## Generate Answers

In [7]:
# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=8192,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

Preprocess dataset

In [29]:
df = pd.read_csv(test_dataset_path)

# Define the schema types
schemas = ["CauseToEffect", "ExpertOpinion", "Analogy", "FearAppeal"]

# Prepare the JSON structure
json_data = []
for idx, row in df.iterrows():
    context = row['input'].strip()
    original_id = row['id']

    for schema in schemas:
        entry = {
            "prompt": [
                {
                    "role": "system",
                    "content": "You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas."
                },
                 {
                    "role": "user",
                    "content": f"Generate one critical question addressing the provided context. Ensure it matches the schema:{schema}\n\nContext: {context}"
                }

            ],
            "id": f"{original_id}_{schema}",
            "schema" : schema

        }
        json_data.append(entry)


# Save the JSON data to a file
output_file = 'processed_dataset.json'
with open(output_file, 'w') as f:
    json.dump(json_data, f, indent=4)

In [32]:
with open('processed_dataset.json', 'r') as file:
    data = json.load(file)

# Prepare inputs in batch mode
prompts = []
prompt_ids = []
for entry in data:
    messages = entry['prompt']
    formatted_messages = [
        {'role': msg['role'], 'content': msg['content']} for msg in messages
    ]
    prompts.append(tokenizer.apply_chat_template(
        formatted_messages,
        tokenize=False,
        add_generation_prompt=True
    ))
    prompt_ids.append(entry['id'])

In [31]:
print(prompts)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate one critical question addressing the provided context. Ensure it matches the schema:CauseToEffect\n\nContext: CLINTON: "which may prove to be an intelligence benefit\r\nwe\'ve got to do everything we can to vacuum up intelligence from Europe, from the Middle East\r\nThat means we\'ve got to work more closely with our allies, and that\'s something that Donald has been very dismissive of\r\nWe\'re working with NATO, the longest military alliance in the history of the world, to really turn our attention to terrorism\r\nWe\'re working with our friends in the Middle East, many of which, as you know, are Muslim majority nations\r\nDonald has consistently insulted Muslims abroad, M

In [33]:
batch_size = 5

# Process in batches to prevent memory overflow
for i in range(0, len(prompts), batch_size):
    batch_prompts = prompts[i:i+batch_size]
    batch_ids = prompt_ids[i:i+batch_size]

    # Tokenize with padding for batch processing
    inputs = tokenizer(batch_prompts, return_tensors='pt', padding=True).to('cuda')

    # Generate outputs
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Print results
    for j, output in enumerate(decoded_outputs):
        print(f"Prompt ID: {batch_ids[j]}")
        print(f"Context: {batch_prompts[j]}")
        print(f"Critical Question: {output.strip()}\n")

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.69 GiB. GPU 0 has a total capacity of 22.16 GiB of which 4.74 GiB is free. Process 35926 has 17.42 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 2.94 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
inputs = tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")

# Generate outputs
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print results
for i, output in enumerate(decoded_outputs):
    print(f"Prompt ID: {data[i]['id']}")
    print(f"Context: {prompts[i]}")
    print(f"Critical Question: {output.strip()}\n")


In [13]:
dataset = load_dataset("json", data_files='processed_dataset.json', split="train")

# Check if the 'prompt' column exists
if 'prompt' not in dataset.column_names:
    raise ValueError("The JSON file must contain a 'prompt' field.")

# Extract the input column as a list of prompts
prompts = []
for item in dataset:
    if isinstance(item['prompt'], list) and item['prompt'][0]['role'] == 'user':
        prompts.append(item['prompt'][0]['content'])

print(prompts)


['Generate one critical question addressing the provided context. Ensure it matches the schema:CauseToEffect\n\nContext: CLINTON: "which may prove to be an intelligence benefit\r\nwe\'ve got to do everything we can to vacuum up intelligence from Europe, from the Middle East\r\nThat means we\'ve got to work more closely with our allies, and that\'s something that Donald has been very dismissive of\r\nWe\'re working with NATO, the longest military alliance in the history of the world, to really turn our attention to terrorism\r\nWe\'re working with our friends in the Middle East, many of which, as you know, are Muslim majority nations\r\nDonald has consistently insulted Muslims abroad, Muslims at home, when we need to be cooperating with Muslim nations and with the American Muslim community\r\nThey\'re on the front lines\r\nThey can provide information to us that we might not get anywhere else\r\nThey need to have close working cooperation with law enforcement in these communities, not b

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

def formatting_prompts_func(prompts_list):
    convos = [{"conversations": [p]} for p in prompts_list]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return texts


# Apply formatting to the prompts
formatted_prompts = formatting_prompts_func(prompts)

In [ ]:
# Tokenize the inputs with padding
inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True).to("cuda")

# Generate outputs
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Prepare the output in the desired JSON format
output_json = {}
for idx, (input_text, cq) in enumerate(zip(prompts, decoded_outputs)):
    output_json[str(idx)] = {
        "input": input_text,
        "cqs": [
            {
                "schema": "FearAppeal",
                "cq": cq.strip()
            }
        ]
    }

# Save the JSON file
with open('/mnt/data/critical_questions.json', 'w') as f:
    json.dump(results_path, f, indent=4)

print("Critical questions generated and saved to critical_questions.json")

In [ ]:
# Import required libraries
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the input CSV file
data = pd.read_csv('/mnt/data/test.csv')

# Extract the input column
prompts = data['input'].tolist()

# Load a smaller, more memory-efficient model
model_name = 'google/flan-t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cpu')

# Prepare for batch processing with reduced memory footprint
batch_size = 4  # Small batch size to prevent OOM errors
results = []

for i in range(0, len(prompts), batch_size):
    batch_prompts = prompts[i:i+batch_size]
    inputs = tokenizer(batch_prompts, return_tensors='pt', padding=True, truncation=True).to('cpu')
    with torch.no_grad():  # Reduce memory usage during generation
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    results.extend(decoded_outputs)

# Combine the results into a DataFrame for better visualization
result_df = pd.DataFrame({'id': data['id'], 'input': prompts, 'critical_question': results})

# Display the results to the user
tools.display_dataframe_to_user('Generated Critical Questions', result_df)

Do this cell only when you load the complete model

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 8192,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [8]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [9]:
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [10]:
def generate_response(prompt_texts, max_new_tokens=64):

    # Format the prompts for the llama-3.1 chat template
    chat_prompts = [ chat_template.format("You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas", prompt) for prompt in prompts]

    print(chat_prompts)

    # Tokenize the inputs with padding
    inputs = tokenizer(chat_prompts, return_tensors="pt", padding=True).to("cuda")

    # Generate outputs
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the outputs
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract the responses after "### Response:" and clean up
    responses = []
    for full_output in decoded_outputs:
        '''
        if "### Response:" in full_output:
            response = full_output.split("### Response:", 1)[-1].strip()
        else:
            response = full_output.strip()

        # Remove leading 'assistant' if present
        if response.lower().startswith("assistant"):
            response = response[len("assistant"):].lstrip("\n ").strip()
'''
        responses.append(full_output)
        print(full_output)
    return responses

In [11]:
'''
def generate_response(prompt_texts, max_new_tokens=64):
    inputs = tokenizer(prompt_texts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    responses = []
    for full_output in decoded:
        # Extract text after ### Response:
        if "### Response:" in full_output:
            response = full_output.split("### Response:")[-1].strip()
        else:
            response = full_output.strip()

        # Remove leading 'assistant\n\n' if present
        if response.lower().startswith("assistant"):
            response = response[len("assistant"):].lstrip("\n ").strip()

        responses.append(response)
    return responses
    '''

'\ndef generate_response(prompt_texts, max_new_tokens=64):\n    inputs = tokenizer(prompt_texts, return_tensors="pt", padding=True, truncation=True).to("cuda")\n\n    outputs = model.generate(\n        **inputs,\n        max_new_tokens=max_new_tokens,\n        do_sample=True,\n        temperature=0.7,\n        top_p=0.9,\n        eos_token_id=tokenizer.eos_token_id,\n    )\n\n    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)\n\n    responses = []\n    for full_output in decoded:\n        # Extract text after ### Response:\n        if "### Response:" in full_output:\n            response = full_output.split("### Response:")[-1].strip()\n        else:\n            response = full_output.strip()\n\n        # Remove leading \'assistant\n\n\' if present\n        if response.lower().startswith("assistant"):\n            response = response[len("assistant"):].lstrip("\n ").strip()\n\n        responses.append(response)\n    return responses\n    '

In [12]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [13]:
schemas = {
"CauseToEffect": """'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?""",

"ExpertOpinion": """'Expert Opinion' with the examples:
Is <expertE> a genuine expert in <domainD>?
Is <eventA> consistent with what other experts in <domainD> say? """,

"Analogy": """'Analogy' with the examples:
Are <C1> and <C2> similar in the respect cited?
Is <eventA> true in <C1>?""",

"FearAppeal": """'Fear Appeal' with the examples:
Is <eventB> bad? Why and to whom is it bad?
Is <eventA> away to prevent <eventB>?"""
}

In [ ]:
results = {}
chunk_size = 20

for chunk in pd.read_csv(test_dataset_path, chunksize=chunk_size):
    contexts = chunk['input'].tolist()
    ids = chunk['id'].tolist()
    prompts = []
    input_ids = []
    schema_ids = []


    for idx, input_text in enumerate(contexts):
        input_id = ids[idx]
        for schema_name, schema_template in schemas.items():
            instruction = f"""### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
{schema_name}

### Context:
{input_text}

Respond with only the generated question.

### Response:

"""

            prompts.append(instruction)
            input_ids.append(input_id)
            schema_ids.append(schema_name)

    batch_size = 5
    num_batches = (len(prompts) + batch_size - 1) // batch_size

    with tqdm(total=num_batches, desc="Generating Critical Questions", leave=True) as pbar:
        for batch_start in range(0, len(prompts), batch_size):
            batch_prompts = prompts[batch_start:batch_start+batch_size]
            batch_outputs = generate_response(batch_prompts, max_new_tokens=256)

            for curr_id, schema_name, output in zip(
                input_ids[batch_start:batch_start + batch_size],
                schema_ids[batch_start:batch_start + batch_size],
                batch_outputs
                ):
                if curr_id not in results:
                    results[curr_id] = {
                        "input": chunk.loc[chunk['id'] == curr_id, 'input'].values[0],
                        "cqs": []
                    }
                results[curr_id]['cqs'].append({
                    "schema": schema_name,
                    "cq": output
                    })

            torch.cuda.empty_cache()
            gc.collect()
            pbar.update(1)

with open(results_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Results saved to {results_path}")


Generating Critical Questions:   0%|          | 0/16 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
assistantassistant

Does trickle-down economics lead to broad-based, inclusive growth in America?
system

Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemasuser

### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
Analogy

### Context:
CLINTON: "it is
trumped-up trickle-down
Trickle-down did not work
It got us into the mess we were in, in 2008 and 2009
Slashing taxes on the wealthy hasn't worked
a lot of really smart, wealthy people know that
they are saying, hey, we need to do more to make the contributions we should be making to rebuild the middle class
I don't think top-down works in America
building the middle class, investing in the middle class, making college debt-free so more young people can get their education, helping people refinance

Generating Critical Questions:   0%|          | 0/16 [00:00<?, ?it/s]

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 23 July 2024\n\nYou generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n### Instruction:\nGenerate one critical question addressing the provided context. Ensure it matches the schema:\nCauseToEffect\n\n### Context:\nCL: "Well, both sides of the transaction suffer as a result of ignoring moral hazard\r\nbecause moral hazard is a natural part of the process of lending and borrowing\r\nAnd if you suppress it, then in fact you\'re in a situation where the wealth that you look like you\'re generating is fool’s gold, not real wealth at all."\n\nRespond with only the generated question.\n\n### Response:\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>', '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 202

## Commit & Push

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabrriga.ch"


commit_message = f"Finetuned generation"
!git add .
!git commit -m "{commit_message}"
!git push